In [76]:
import pandas as pd
import numpy as np
import json
from datetime import date, timedelta
import urllib
import pickle
import xgboost as xgb
# Для рейтинга команд
# создать мапу с старыми и новыми именами колонок для рейтинга команд
def name_columns(z1, elo):
    mapa = {}
    for index, key in enumerate(z1.keys()):
        mapa[key] = key + elo
    return mapa
# вытащить все данные по рейтингу (elo32, elo64, glicko, glicko2) одной команды
def get_data_team(data):
    ratings = ['elo32', 'elo64', 'glicko', 'glicko2']

    #Создать колонку с именем команды
    team = pd.DataFrame(columns=['team_Name'])
    team['team_Name'] = [data.get('teamName')]

    for rat in ratings:
        current_rating = data.get(rat)
        columns = name_columns(current_rating, '_' + rat)
        df = pd.DataFrame(data.get(rat), index=range(0,1))     
        df.rename(columns=columns, inplace=True)
        team = pd.DataFrame.merge(team, df, left_index=True, right_index=True)
    return team

#Для KDA Героевg
# Вытащить КДА по всем героям одной команды (нужна ссылка на сайт и массив из 5 героев одной команды)
def get_KDA(url, array_heroes):
    columns = ['hero', 'kills', 'deaths', 'assists', 'kda', 'avgKal', 'gpm', 'xpm', 'lastHits', 'denies', 'level',
           'heroDamage', 'towerDamage', 'heroHealing', 'goldSpent']
    # ДФ для соединения всех герове матча в один ДФ 
    df_heroe_match_teams = pd.DataFrame()

    # выгрзить json с предыдущей ссылки
    sauce = urllib.request.Request(url, headers={'User-agent': 'Mozilla/5.0'} )
    dat = json.loads(urllib.request.urlopen(sauce).read().decode()).get('data')
    df_data_tabel_for_heroes_radiant = pd.DataFrame(dat, columns=columns)
    df_kda_heroes = []
    for her in array_heroes:
        # достать KDA по каждому герою и убрать стобец с индексом героя
        kda_heroe = df_data_tabel_for_heroes_radiant[df_data_tabel_for_heroes_radiant['hero'] == her]
        kda_heroe = kda_heroe.drop('hero', axis=1)
        if len(kda_heroe) == 0:
            kda_heroe = [0,0,0,0,0,0,0,0,0,0,0,0,0,0]
        df_kda_heroes = np.append(df_kda_heroes, kda_heroe)
    return df_kda_heroes

# Head_to_head
# добавить в массив суммарный показатель elo для героя vs всх врагов
def elo_heroes_vs_enemies(array_heroes, array_enemies, df_elo_herVsEne):
    df_elo = []
    # две переменные для записи суммарного elo каждой команды
    for her in array_heroes:
        hero_elo = 0
        # суммировать elo героя против героев противника
        for her_enemy in array_enemies:
            try:
                elo =  df_elo_herVsEne['shift'][df_elo_herVsEne['hero'] == 
                                              her][df_elo_herVsEne['againstHero'] == her_enemy].item()
            except:
                elo=0
            hero_elo += elo
        # записать в основной массив elo по каждому герою
        df_elo = np.append(df_elo, hero_elo)
    return df_elo
# создать массив elo по героям имея только ссылку на сайт
def get_elo_head_to_head(url):
    # выгрзить json с предыдущей ссылки и создать ДФ
    sauce = urllib.request.Request(url, headers={'User-agent': 'Mozilla/5.0'} )
    dat = json.loads(urllib.request.urlopen(sauce).read().decode()).get('data')
    df_elo_herVsEne = pd.DataFrame(dat)
    
    radiant_eloVsEnemy = elo_heroes_vs_enemies(radiant, dire, df_elo_herVsEne)
    dire_eloVsEnemy = elo_heroes_vs_enemies(dire, radiant, df_elo_herVsEne)
    return np.append(radiant_eloVsEnemy, dire_eloVsEnemy)

#AvgElo
# вытащить ДФы для одного, пары, тройки героев из сайта по дате
def get_df_avgElo_heroes(df):
    # создать ДФ для одного героя, пары и тройки
    df_one = df.loc[(index for index, x in enumerate(df['heroes']) if len(x) == 1), :]  
    df_one = df_one.reset_index().drop('index', axis=1)
#     df_double = df.loc[(index for index, x in enumerate(df['heroes']) if len(x) == 2), :]  
#     df_triple = df.loc[(index for index, x in enumerate(df['heroes']) if len(x) == 3), :]  

    return df_one#, df_double, df_triple

# вытащить таблицу с elo героев для команды 
def get_df_AvgElo_heroes_teams(id_radiant_or_dire):
    # создать ссылку с данными по Avg.Elo для двух предыдущих месяцев игры
    url = ('http://www.datdota.com/api/heroes/elo?teams={}&tier=1&valve-event=does-not-matter&threshold=5'.format(id_radiant_or_dire) +
        '&patch=7.09&patch=7.08&patch=7.07&patch=7.06&patch=7.05&patch=7.04&patch=7.03&patch=7.02' + 
        '&patch=7.01&patch=7.00&patch=6.88&patch=6.87&patch=6.86&winner=either'+
        '&after=01%2F01%2F2011'+ 
        '&before={}%2F{}%2F{}'.format(date.today().day, date.today().month, date.today().year) + 
        '&duration=0%3B200&duration-value-from=0&duration-value-to=200')
    # выгрузить все с сайта и создать ДФ
    sauce = urllib.request.Request(url, headers={'User-agent': 'Mozilla/5.0'})
    dat = json.loads(urllib.request.urlopen(sauce).read().decode()).get('data')
    df_url = pd.DataFrame(dat)
    
    # создать ДФ для соло героев за все время игр команды (сигнатурки)
    return get_df_avgElo_heroes(df_url)

# Исходные данные

In [77]:
list_heroes = pd.read_csv('../All_Heroes.csv')
(np.array(list_heroes['localized_name']))

array(['Anti-Mage', 'Axe', 'Bane', 'Bloodseeker', 'Crystal Maiden',
       'Drow Ranger', 'Earthshaker', 'Juggernaut', 'Mirana', 'Morphling',
       'Shadow Fiend', 'Phantom Lancer', 'Puck', 'Pudge', 'Razor',
       'Sand King', 'Storm Spirit', 'Sven', 'Tiny', 'Vengeful Spirit',
       'Windranger', 'Zeus', 'Kunkka', 'Lina', 'Lion', 'Shadow Shaman',
       'Slardar', 'Tidehunter', 'Witch Doctor', 'Lich', 'Riki', 'Enigma',
       'Tinker', 'Sniper', 'Necrophos', 'Warlock', 'Beastmaster',
       'Queen of Pain', 'Venomancer', 'Faceless Void', 'Wraith King',
       'Death Prophet', 'Phantom Assassin', 'Pugna', 'Templar Assassin',
       'Viper', 'Luna', 'Dragon Knight', 'Dazzle', 'Clockwerk', 'Leshrac',
       "Nature's Prophet", 'Lifestealer', 'Dark Seer', 'Clinkz',
       'Omniknight', 'Enchantress', 'Huskar', 'Night Stalker',
       'Broodmother', 'Bounty Hunter', 'Weaver', 'Jakiro', 'Batrider',
       'Chen', 'Spectre', 'Ancient Apparition', 'Doom', 'Ursa',
       'Spirit Breaker', 'G

In [92]:
radiant_team_name =     'iG.Vitality'
dire_team_name =   'EHOME'

radiant_team_id =   2640025
dire_team_id =     4

radiant_H1 = list_heroes['id'][list_heroes['localized_name'] == "Witch Doctor"].item()
radiant_H2 = list_heroes['id'][list_heroes['localized_name'] == "Sand King"].item()
radiant_H3 = list_heroes['id'][list_heroes['localized_name'] == "Juggernaut"].item()
radiant_H4 = list_heroes['id'][list_heroes['localized_name'] == "Shadow Fiend"].item()
radiant_H5 = list_heroes['id'][list_heroes['localized_name'] == "Tidehunter"].item()
dire_H1 = list_heroes['id'][list_heroes['localized_name'] == "Chen"].item()
dire_H2 = list_heroes['id'][list_heroes['localized_name'] == "Batrider"].item()
dire_H3 = list_heroes['id'][list_heroes['localized_name'] == "Lion"].item()
dire_H4 = list_heroes['id'][list_heroes['localized_name'] == "Queen of Pain"].item()
dire_H5 = list_heroes['id'][list_heroes['localized_name'] == "Lone Druid"].item()

radiant = [radiant_H1, radiant_H2, radiant_H3, radiant_H4, radiant_H5]
dire = [dire_H1, dire_H2, dire_H3, dire_H4, dire_H5]

# Сбор всех таблиц
# Рейтинг команд

In [93]:
# ВЫТАЩИТЬ из таблицы рейтинга пару героев
# создать правильную ссылку для посика героев по команде radiant
url = ('https://www.datdota.com/api/ratings')

# выгрзить json с предыдущей ссылки и создать ДФ
sauce = urllib.request.Request(url, headers={'User-agent': 'Mozilla/5.0'})
dat = json.loads(urllib.request.urlopen(sauce).read().decode())
data = dat.get('data')
# создать DF для сохранения
all_teams = pd.DataFrame()
for i in data:
    # убрать повторяющиеся команды (DC и Infamous)
    if (i.get('valveId') != 2512249) and (i.get('valveId') != 2672298):
        team = get_data_team(i) 
        all_teams = pd.concat([all_teams, team], ignore_index=True)
all_teams = all_teams.drop(['phi_glicko', 'sigma_glicko2'],  axis=1)

# вытащить рейтинги двух команд и убрать имя
radiant_rating_df = all_teams[all_teams['team_Name'] == radiant_team_name]

print ((len(radiant_rating_df.index)))
radiant_rating_df = radiant_rating_df.drop(['team_Name'],  axis=1)

dire_rating_df = all_teams[all_teams['team_Name'] == dire_team_name]
print ((len(dire_rating_df.index)))

dire_rating_df = dire_rating_df.drop(['team_Name'],  axis=1)
# соединить в один массив
df_rating_teams = np.append(radiant_rating_df, dire_rating_df )

print (len(df_rating_teams))


1
1
36


# KDA для героев за все время с патча 7,00

In [94]:
url_kda_Heroes = ('http://www.datdota.com/api/heroes/performances?patch=7.10&patch=7.09&patch=7.08&patch=7.07'+
                    '&patch=7.06&patch=7.05&patch=7.04&patch=7.03&patch=7.02&patch=7.01&patch=7.00'+
                  '&winner=either&after=01%2F01%2F2011&before=09%2F03%2F2018&duration=0%3B200'+
                  '&duration-value-from=0&duration-value-to=200&tier=1&valve-event=does-not-matter&threshold=10')
radiant_kdaHeroes = get_KDA(url_kda_Heroes, radiant)
dire_kdaHeroes = get_KDA(url_kda_Heroes, dire)
# соединить всех героев в один массив
df_kda_heroes = np.append(radiant_kdaHeroes, dire_kdaHeroes)
print (len(df_kda_heroes))

140


# KDA для героев по каждой команды (как команды отыгрывают на этих героях)

In [95]:
# Radiant
url_heroes_radiant =('http://www.datdota.com/api/heroes/performances?teams={}'.format(radiant_team_id) + 
                     '&patch=7.10&patch=7.09&patch=7.08&patch=7.07&patch=7.06&patch=7.05&patch=7.04&patch=7.03'+
                     '&patch=7.02&patch=7.01&patch=7.00&winner=either&after=01%2F01%2F2011&before=09%2F03%2F2018'+
                     '&duration=0%3B200&duration-value-from=0&duration-value-to=200&tier=1&valve-event='+
                     'does-not-matter&threshold=1') 
radiant_kdaHeroes = get_KDA(url_heroes_radiant, radiant)
#Dire
url_heroes_dire =('http://www.datdota.com/api/heroes/performances?teams={}'.format(dire_team_id) + 
                  '&patch=7.10&patch=7.09&patch=7.08&patch=7.07&patch=7.06&patch=7.05&patch=7.04&patch=7.03'+
                  '&patch=7.02&patch=7.01&patch=7.00&winner=either&after=01%2F01%2F2011&before=09%2F03%2F2018&'+
                  'duration=0%3B200&duration-value-from=0&duration-value-to=200&tier=1&valve-event=does-not-matter'+
                  '&threshold=1') 
dire_kdaHeroes = get_KDA(url_heroes_dire, dire)
print (len(radiant_kdaHeroes))
print (len(dire_kdaHeroes))
df_kda_heroes_teams = np.append(radiant_kdaHeroes, dire_kdaHeroes)
df_kda_heroes_teams = np.nan_to_num(df_kda_heroes_teams)

70
70


# Elo героя относительно его врагов (Head-to_head)

In [96]:
url_Head_to_head_allPatch = ('http://www.datdota.com/api/heroes/head-to-head-elo?tier=1&valve-event='+
                             'does-not-matter&threshold=1&patch=7.10&patch=7.09&patch=7.08&patch=7.07&patch=7.06'+
                             '&patch=7.05&patch=7.04&patch=7.03&patch=7.02&patch=7.01&patch=7.00&patch=6.88&patch=6.87&'+
                             'patch=6.86&patch=6.85&patch=6.84&patch=6.83&patch=6.82&patch=6.81&patch=6.80&patch=6.79&'+
                             'patch=6.78&patch=6.77&patch=6.76&patch=6.75&patch=6.74&winner=either&after=01%2F01%2F2011&'+
                             'before=09%2F03%2F2018&duration=0%3B200&duration-value-from=0&duration-value-to=200')
head_to_head_allPatch = get_elo_head_to_head(url_Head_to_head_allPatch)

url_Head_to_head_700Patch = ('http://www.datdota.com/api/heroes/head-to-head-elo?tier=1&valve-event=does-not-matter&'+
                            'threshold=1&patch=7.10&patch=7.09&patch=7.08&patch=7.07&patch=7.06&patch=7.05&patch=7.04&patch=7.03&patch=7.02&'+
                            'patch=7.01&patch=7.00&winner=either&after=01%2F01%2F2011&before=09%2F03%2F2018&duration=0%3B200&'+
                            'duration-value-from=0&duration-value-to=200')
head_to_head_700Patch = get_elo_head_to_head(url_Head_to_head_700Patch)
print (len(head_to_head_allPatch))
print (len(head_to_head_700Patch))

10
10


# Пока без типов героев Так как мало влияют

# Метовые герои AvgElo

In [97]:
# ДФ для соединения всех герове матча в один ДФ 
df_heroes_match_AvgElo = []
# создать дату два месяца назад от даты матча
two_month_ago = date.today() - timedelta(60)
# создать ссылку с данными по Avg.Elo для двух предыдущих месяцев игры
url = ('http://www.datdota.com/api/heroes/elo?tier=1&valve-event=does-not-matter&threshold=10' +
    '&patch=7.09&patch=7.08&patch=7.07&patch=7.06&patch=7.05&patch=7.04&patch=7.03&patch=7.02' + 
    '&patch=7.01&patch=7.00&patch=6.88&patch=6.87&patch=6.86&winner=either'+
    '&after={}%2F{}%2F{}'.format(two_month_ago.day, two_month_ago.month, two_month_ago.year) + 
    '&before={}%2F{}%2F{}'.format(date.today().day, date.today().month, date.today().year) + 
    '&duration=0%3B200&duration-value-from=0&duration-value-to=200')
# выгрузить все с сайта и создать ДФ
sauce = urllib.request.Request(url, headers={'User-agent': 'Mozilla/5.0'})
dat = json.loads(urllib.request.urlopen(sauce).read().decode()).get('data')
df_url = pd.DataFrame(dat)
# создать ДФ для одного героя за 2 предыдущих месяца
df_data_tabel_for_heroes = get_df_avgElo_heroes(df_url)

for her in (radiant + dire):
    # вытащить avg elo для данного героя
    avgElo_hero = df_data_tabel_for_heroes.loc[(index for index, x in enumerate(
                                        df_data_tabel_for_heroes['heroes']) if x == [her]),'eloShift']

    # проверить есть ли герой
    try:
        avgElo_hero = float(avgElo_hero)
    except:
        avgElo_hero = 0
    df_heroes_match_AvgElo.append(avgElo_hero)
print(len(df_heroes_match_AvgElo))

10


In [98]:
# ДФ для соединения всех герове матча в один ДФ 
df_heroes_match_radiant = []
df_heroes_match_dire = []
# создать дату два месяца назад от даты матча
two_month_ago = date.today() - timedelta(60)

# RADIANT
# создать ДФ для одного героя за 2 предыдущих месяца
df_data_tabel_radiant = get_df_AvgElo_heroes_teams(radiant_team_id)

for her in (radiant):
    # вытащить avg elo для данного героя
    avgElo_hero = df_data_tabel_radiant.loc[(index for index, x in enumerate(
                                        df_data_tabel_radiant['heroes']) if x == [her]),'eloShift']
    # проверить есть ли герой
    try:
        avgElo_hero = float(avgElo_hero)
    except:
        avgElo_hero = 0
    df_heroes_match_radiant.append(avgElo_hero)

#DIRE
# создать ДФ для одного героя за 2 предыдущих месяца
df_data_tabel_dire = get_df_AvgElo_heroes_teams(dire_team_id)

for her in (dire):
    # вытащить avg elo для данного героя
    avgElo_hero = df_data_tabel_dire.loc[(index for index, x in enumerate(
                                        df_data_tabel_dire['heroes']) if x == [her]),'eloShift']
    # проверить есть ли герой
    try:
        avgElo_hero = float(avgElo_hero)
    except:
        avgElo_hero = 0
    df_heroes_match_dire.append(avgElo_hero)
df_heroes_teams_AvgElo = np.append(df_heroes_match_radiant, df_heroes_match_dire)
print (len(df_heroes_teams_AvgElo))

10


# Соединить все в один массив для предсказания

In [99]:
# Для данныз из head_to_head по всем патчам
main_array_allPatch = np.append(df_rating_teams, df_kda_heroes)
main_array_allPatch = np.append(main_array_allPatch, df_kda_heroes_teams)
main_array_allPatch = np.append(main_array_allPatch, head_to_head_allPatch)
main_array_allPatch = np.append(main_array_allPatch, df_heroes_match_AvgElo)
main_array_allPatch = np.append(main_array_allPatch, df_heroes_teams_AvgElo)


# Для данныз из head_to_head с патча 7,00
main_array_700Patch = np.append(df_rating_teams, df_kda_heroes)
main_array_700Patch = np.append(main_array_700Patch, df_kda_heroes_teams)
main_array_700Patch = np.append(main_array_700Patch, head_to_head_700Patch)
main_array_700Patch = np.append(main_array_700Patch, df_heroes_match_AvgElo)
main_array_700Patch = np.append(main_array_700Patch, df_heroes_teams_AvgElo)
len(main_array_allPatch)


346

In [100]:
filename = 'grid_search_GB_model_allData_dayMatch.sav'
test_gb = pickle.load(open(filename, 'rb'))
print('Модель обученая На всех патчах до дня матча')
print (test_gb.predict([main_array_allPatch]))
print (test_gb.predict_proba([main_array_allPatch]))

filename = 'grid_search_GB_model_allData_2018.05.06.sav'
test_gb = pickle.load(open(filename, 'rb'))
print('Модель обученая На всех патчах до 2018.05.06')
print (test_gb.predict([main_array_allPatch]))
print (test_gb.predict_proba([main_array_allPatch]))


Модель обученая На всех патчах до дня матча
[1]
[[ 0.05853994  0.94146006]]
Модель обученая На всех патчах до 2018.05.06
[1]
[[ 0.33351526  0.66648474]]
